## Reading Input

In [1]:
import pandas as pd
from proj_utils.input_interpreter import format_input

In [2]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()

In [3]:
print(routes)

[2946091 2922001 2939484 2958047 2970877 2990001 3005971 2604001 3016355
 3027038 3044702]


In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance

from proj_utils.path import print_path

from solver_step import SolverStep
%matplotlib widget

# Simulated Annealing Solution

In [2]:
import simulated_annealing.solver as SA # Simulated anealing solver

In [3]:
#dists, demand, points = format_input(routes[2],customer_df,deposit_df)
dists, demand, points = generator(20)

In [4]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,(30,20),demand)) else float('inf')
solver_step = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [44]:
init_state = nearest_neighbour_solution(dists, demand, (30,20))
init_step = SolverStep(0,init_state, eval_function(init_state),100)
step_1 = solver_step(init_step)
print(step_1.get_best_sol())
print(step_1.get_best_sol_value())

[0, 13, 1, 8, 11, 18, 5, 15, 3, 6, 16, 12, 2, 9, 17, 10, 4, 14, 19, 7, 0]
477.35961388699036


In [45]:
step = step_1

In [108]:
step = solver_step(step)
print(step.get_best_sol())
print(step.get_best_sol_value())

[0, 7, 6, 3, 18, 5, 15, 11, 8, 9, 10, 4, 13, 1, 17, 2, 12, 16, 19, 14, 0]
437.58038294712327


# Taboo Search

In [109]:
import taboo_search.solver as TS

In [110]:
dists, demand, points = generator(20)

In [111]:
taboo_list_size = 10
solver_step = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    lambda x: calculate_path_distance(x, dists) if(valid_path(x,(30,20),demand)) else float('inf'),
    taboo_list_size
    )

In [113]:
init_state = nearest_neighbour_solution(dists, demand, (30,20))
init_step = SolverStep(0,init_state, eval_function(init_state),2)
step_1 = solver_step(init_step)
print(step_1.get_best_sol())

[0, 1, 6, 19, 11, 13, 14, 18, 4, 15, 3, 8, 16, 12, 9, 5, 7, 10, 2, 17, 0]


In [114]:
step = step_1

In [139]:
step = solver_step(step)
print(step.get_best_sol())
print(step.get_best_sol_value())

[0, 15, 18, 4, 14, 13, 1, 11, 19, 6, 8, 3, 16, 12, 5, 9, 7, 17, 2, 10, 0]
477.70388942476717
